In [129]:
import os
import pandas as pd
import numpy as np
from tqdm import tqdm

In [183]:
N_GRAM = 2

# Lendo arquivo

Lendo arquivo CETEN.xml e adicionando _dummy characters_ nas linhas com o tamanho do n-gram

- `<s>`: início da sentença
- `</s>`: fim da sentença

In [46]:
def process_line(line, n_gram=3):
    line = line.replace('\n', '').strip()
    line = line.replace('<s> ', '<s> '*(n_gram-1))
    line = line.replace(' </s>', ' </s>'*(n_gram-1))
    return tuple(line.split(' '))

In [172]:
filepath = './data/ceten.xml'
lines = []
n_lines = -1
i = 0
with open(filepath) as f:
    for line in tqdm(f.readlines()[:n_lines]):
        line = process_line(line, N_GRAM+1)
        lines.append(line)

100%|██████████| 1542880/1542880 [00:19<00:00, 78770.41it/s]


In [174]:
lines[0]

('<s>', '<s>', 'pt', 'no', 'governo', '</s>', '</s>')

## Definindo vocabulário

In [176]:
%%time
def get_vocabulary(lines):
    vocabulary = {}
    for sentence in tqdm(lines):
        for word in sentence:
            if word not in ['<s>', '</s>']:
                vocabulary[word] = vocabulary.get(word, 0) + 1
    return vocabulary

vocabulary = get_vocabulary(lines)
print ('Vocabulary size:', len(vocabulary))

100%|██████████| 1542880/1542880 [00:25<00:00, 60122.71it/s]


Vocabulary size: 214439
CPU times: user 19.1 s, sys: 4.14 s, total: 23.3 s
Wall time: 30.9 s


# Extraindo N-grams

Os n-gramas de um texto são todas as sentenças formadas por `N` palavras

In [180]:
def get_n_grams(lines, n=3):
    n_grams = {}
    for sentence in tqdm(lines):
        sentence_length = len(sentence)
        start_index = 0
        end_index = start_index + n
        while end_index <= sentence_length:

            n_gram = sentence[start_index:end_index]
            n_grams[n_gram] = n_grams.get(n_gram, 0) + 1

            start_index +=1
            end_index = start_index + n

    return n_grams

sentence = [
    ('<s>', 'hoje', 'choveu', 'muito', 'no', 'rio', 'de', 'janeiro', '</s>'),
    ('<s>', 'hoje', 'choveu', 'muito', 'no', 'espírito', 'santo', '</s>'),
    ('<s>', 'hoje', 'nevou', 'muito', 'na', 'bahía', '</s>'),
]
get_n_grams(sentence, n=2)

100%|██████████| 3/3 [00:00<00:00, 4837.72it/s]


{('<s>', 'hoje'): 3,
 ('hoje', 'choveu'): 2,
 ('choveu', 'muito'): 2,
 ('muito', 'no'): 2,
 ('no', 'rio'): 1,
 ('rio', 'de'): 1,
 ('de', 'janeiro'): 1,
 ('janeiro', '</s>'): 1,
 ('no', 'espírito'): 1,
 ('espírito', 'santo'): 1,
 ('santo', '</s>'): 1,
 ('hoje', 'nevou'): 1,
 ('nevou', 'muito'): 1,
 ('muito', 'na'): 1,
 ('na', 'bahía'): 1,
 ('bahía', '</s>'): 1}

In [181]:
n_grams = get_n_grams(lines, n=N_GRAM)

100%|██████████| 1542880/1542880 [01:56<00:00, 13254.08it/s]


In [184]:
n_plus1_grams = get_n_grams(lines, n=N_GRAM+1)

100%|██████████| 1542880/1542880 [01:18<00:00, 19554.63it/s]


# Calculando a _Forward Probability_

$$P(word|sentence) = \frac{Count(sentence+word)}{Count(sentence)}$$

In [190]:
def get_forward_probability(sentence, word, n_grams, n_plus1_grams, vocabulary):
    if word not in vocabulary.keys():
        print (f'Word {word} not in vocabulary')
        return 0

    full_sentence = (*sentence, word)
    sentence_occurrences = n_grams.get(sentence, 0)
    full_sentence_occurences = n_plus1_grams.get(full_sentence, 0)

    print (f'Ocorrências da sentença {sentence}: {sentence_occurrences}')
    print (f'Ocorrências da sentença {full_sentence}: {full_sentence_occurences}')

    if sentence_occurrences == 0:
        return 0

    return full_sentence_occurences/sentence_occurrences

sentence = ('pesquisa', 'datafolha')
next_word = 'revela'
prob = get_forward_probability(sentence, next_word, n_grams, n_plus1_grams, vocabulary)
prob

Ocorrências da sentença ('pesquisa', 'datafolha'): 587
Ocorrências da sentença ('pesquisa', 'datafolha', 'revela'): 9


0.015332197614991482

In [1]:
('pesquisa', 'datafolha', 'revela')[:-1]

('pesquisa', 'datafolha')

In [197]:
def get_next_words(sentence, n_grams, n_plus1_grams):
    full_sentences = {}
    if n_grams.get(sentence, 0) == 0:
        print (f'Sentença {sentence} não encontrada')
        return full_sentences

    for full_sentence in tqdm(n_plus1_grams.keys()):
        if full_sentence[:-1] == sentence:
            full_sentences[full_sentence] = n_plus1_grams.get(full_sentence)

    return full_sentences

sentence = ('pesquisa', 'datafolha')
next_words = get_next_words(sentence, n_grams, n_plus1_grams)
next_words


{('pesquisa', 'datafolha', 'publicada'): 61,
 ('pesquisa', 'datafolha', 'deve'): 1,
 ('pesquisa', 'datafolha', 'divulgada'): 25,
 ('pesquisa', 'datafolha', 'é'): 35,
 ('pesquisa', 'datafolha', 'debateriam'): 3,
 ('pesquisa', 'datafolha', 'realizada'): 71,
 ('pesquisa', 'datafolha', '</s>'): 69,
 ('pesquisa', 'datafolha', 'sobre'): 26,
 ('pesquisa', 'datafolha', 'enéas'): 1,
 ('pesquisa', 'datafolha', 'foi'): 9,
 ('pesquisa', 'datafolha', 'feita'): 30,
 ('pesquisa', 'datafolha', 'a'): 6,
 ('pesquisa', 'datafolha', 'mostra'): 26,
 ('pesquisa', 'datafolha', 'revela'): 9,
 ('pesquisa', 'datafolha', 'traz'): 2,
 ('pesquisa', 'datafolha', 'de'): 24,
 ('pesquisa', 'datafolha', 'aponta'): 4,
 ('pesquisa', 'datafolha', 'dos'): 3,
 ('pesquisa', 'datafolha', 'indica'): 8,
 ('pesquisa', 'datafolha', 'junto'): 2,
 ('pesquisa', 'datafolha', 'ele'): 4,
 ('pesquisa', 'datafolha', 'entre'): 4,
 ('pesquisa', 'datafolha', 'luiz'): 1,
 ('pesquisa', 'datafolha', 'lhe'): 2,
 ('pesquisa', 'datafolha', 'no'):

In [198]:
def get_previous_words(sentence, n_grams, n_plus1_grams):
    full_sentences = {}
    if n_grams.get(sentence, 0) == 0:
        print (f'Sentença {sentence} não encontrada')
        return full_sentences

    for full_sentence in tqdm(n_plus1_grams.keys()):
        if full_sentence[1:] == sentence:
            full_sentences[full_sentence] = n_plus1_grams.get(full_sentence)

    return full_sentences

sentence = ('pesquisa', 'datafolha')
get_previous_words(sentence, n_grams, n_plus1_grams)


100%|██████████| 11475645/11475645 [00:09<00:00, 1204350.08it/s]


{('brasília', 'pesquisa', 'datafolha'): 4,
 ('última', 'pesquisa', 'datafolha'): 94,
 ('na', 'pesquisa', 'datafolha'): 53,
 ('da', 'pesquisa', 'datafolha'): 46,
 ('segundo', 'pesquisa', 'datafolha'): 67,
 ('de', 'pesquisa', 'datafolha'): 26,
 ('nova', 'pesquisa', 'datafolha'): 3,
 ('a', 'pesquisa', 'datafolha'): 119,
 ('pela', 'pesquisa', 'datafolha'): 9,
 ('em', 'pesquisa', 'datafolha'): 14,
 ('namorados', 'pesquisa', 'datafolha'): 1,
 ('conforme', 'pesquisa', 'datafolha'): 5,
 ('<s>', 'pesquisa', 'datafolha'): 77,
 ('lidera', 'pesquisa', 'datafolha'): 1,
 ('hoje', 'pesquisa', 'datafolha'): 1,
 ('recente', 'pesquisa', 'datafolha'): 18,
 ('fonte', 'pesquisa', 'datafolha'): 3,
 ('com', 'pesquisa', 'datafolha'): 4,
 ('primeira', 'pesquisa', 'datafolha'): 5,
 ('paulo', 'pesquisa', 'datafolha'): 1,
 ('útlima', 'pesquisa', 'datafolha'): 1,
 ('uma', 'pesquisa', 'datafolha'): 7,
 ('à', 'pesquisa', 'datafolha'): 3,
 ('econômica', 'pesquisa', 'datafolha'): 1,
 ('apura', 'pesquisa', 'datafolha')

# Calculando a _backward probability_

$$P(word|sentence) = \frac{Count(word+sentence)}{Count(sentence)}$$

In [166]:
def get_backward_probability(sentence, word, n_grams, n_plus1_grams, vocabulary):
    if word not in vocabulary.keys():
        print (f'Word {word} not in vocabulary')
        return 0
    full_sentence = (word, *sentence)
    sentence_occurrences = n_grams.get(sentence, 0)
    full_sentence_occurences = n_plus1_grams.get(full_sentence, 0)

    print (f'Ocorrências da sentença {sentence}: {sentence_occurrences}')
    print (f'Ocorrências da sentença {full_sentence}: {full_sentence_occurences}')

    if sentence_occurrences == 0:
        return 0

    return full_sentence_occurences/sentence_occurrences

sentence = ('datafolha', 'publicada', 'hoje')
previous_word = 'pesquisa'
prob = get_backward_probability(sentence, previous_word, n_grams, n_plus1_grams, vocabulary)
prob

Ocorrências da sentença ('datafolha', 'publicada', 'hoje'): 24
Ocorrências da sentença ('pesquisa', 'datafolha', 'publicada', 'hoje'): 20


0.8333333333333334

In [103]:
n_plus1_grams.get(('brasília', 'pesquisa', 'datafolha', 'publicada', 'hoje', 'revela'))

1

In [167]:
df_sentences = pd.read_excel('sentences.xlsx', engine='openpyxl')
df_sentences.tail()

,target,questions,corrAnswer,sentID
199,###### Por enquanto devemos continuar com noss...,NaN,-,200
200,###### Não consigo imaginar sobre o que mais v...,NaN,-,201
201,###### A porta foi fechada atrás dele e Sérgio...,Havia outros três homens na cela com Sérgio?,não,202
202,###### Caroline pegou os pratos vazios da mesa...,Caroline colocou os pratos vazios na pia?,não,203
203,###### De manhã era bom se levantar e não ter ...,NaN,-,204


In [169]:
def get_ngrams(sentence, n):
    return tuple(sentence.split(' ')[-n:])
df_sentences['n_grams'] = df_sentences['target'].apply(lambda x: get_ngrams(x, n_gram))
df_sentences[['target', 'n_grams']]

,target,n_grams
0,###### Ana se descontrolou e riu,"(descontrolou, e, riu)"
1,###### José escreveu no envelope,"(escreveu, no, envelope)"
2,###### Ele ficou encostado na parede,"(encostado, na, parede)"
3,###### Helena correu para o banheiro,"(para, o, banheiro)"
4,###### Rogério sorriu e sentou-se,"(sorriu, e, sentou-se)"
...,...,...
199,###### Por enquanto devemos continuar com noss...,"(como, costumávamos, fazer)"
200,###### Não consigo imaginar sobre o que mais v...,"(que, não, falaria)"
201,###### A porta foi fechada atrás dele e Sérgio...,"(homens, na, cela)"
202,###### Caroline pegou os pratos vazios da mesa...,"(migalhas, da, mesa)"


In [171]:
sentence = ('por', 'enquanto', 'devemos')
next_word = 'continuar'
prob = get_forward_probability(sentence, next_word, n_grams, n_plus1_grams, vocabulary)
prob

Ocorrências da sentença ('por', 'enquanto', 'devemos'): 0
Ocorrências da sentença ('por', 'enquanto', 'devemos', 'continuar'): 0


0